# Analyzing image texture using pyCERR


This notebook demonstrates how to filter images using pyCERR.

Supported filters include: *Mean*, *Sobel*, *Laplacian of Gaussian (LoG)*, *Laws'/Laws' Energy*, and *Gabor*.  
All filters are implemented as per the [IBSI2](https://theibsi.github.io/ibsi2/) standard, including support for 3D approximations and rotation invariance where defined.

## Install pyCERR (dev version)

In [1]:
%% capture
! pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR"

## Read sample DICOM images

* The dataset used here, comprising a Lung CT scan with tumor segmentation, was made available by Lambin et al. via CancerData and is included with pyCERR.

* Selected filter parameters used in compatibilty tests with the IBSI-2 standard are used in this demonstration.

In [2]:
import os, numpy as np
from cerr import datasets, plan_container as pc

# Path to DICOM data
sampleDataPath = os.path.dirname(datasets.__file__)
sampleDataDir = os.path.join(sampleDataPath, 'radiomics_phantom_dicom', 'PAT1')

# Import contents to planC
planC = pc.loadDcmDir(sampleDataDir)

('PAT1', 'PAT1', '1.3.6.1.4.1.9590.100.1.2.96328687310426543129572151154132284399', '1.3.6.1.4.1.9590.100.1.2.258301620411152643708006163321128526885', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT')
('PAT1', 'PAT1', '1.3.6.1.4.1.9590.100.1.2.96328687310426543129572151154132284399', '1.3.6.1.4.1.9590.100.1.2.296658988911737913102339329841519593982', 'CT', 'CT', 'CT', 'CT', 'CT', 'CT', 'CT')


## Visualize input scan and tumor mask

In [3]:
from cerr import viewer as vwr

#Display scan and tumor mask
scanNum = 0
strNum = 0
doseNum = []

# Extract scan array
scan3M = planC.scan[scanNum].getScanArray()
scan3M = scan3M.astype(float)

# Visualize in Napari 
viewer, scan_layer, struct_layer, dose_layer, dvf_layer = \
    vwr.showNapari(planC, scan_nums=scanNum, struct_nums=strNum, dose_nums=[], vectors_dict={}, displayMode='2d')


## Example-1 : Mean filter

*Note*:
* Sample filter settings can be downloaded from [https://github.com/cerr/pyCERR-Notebooks](https://github.com/cerr/pyCERR-Notebooks/settings/). The `settingsDir` variable below should be edited to point to your local copy of the settings file.
* The examples below use the *Napari* package for visualization. *Napari* requires a display for rendering graphics and does not support visualization on a remote server (e.g. Google Colab).

### Review filter settings

In [4]:
import json, jsbeautifier
from cerr.radiomics import textureUtils 

options = jsbeautifier.default_options()
options.indent_size = 4

# Path to JSON settings file with filter parameters
settingsFile = 'C:\settings\convolutional_filter_settings\mean_filter.json'  #Replace with Path to sample mean filter settings file
paramS, __ = textureUtils.loadSettingsFromFile(settingsFile)

# Display settings
print(jsbeautifier.beautify(json.dumps(paramS), options))

{
    "structures": ["GTV"],
    "imageType": {
        "Mean": {
            "KernelSize": [5, 5, 5]
        }
    },
    "settings": {
        "padding": {
            "method": "expand",
            "size": [2, 2, 2]
        }
    }
}


In [5]:
# Compute filter response
planC = textureUtils.generateTextureMapFromPlanC(planC, scanNum, strNum, settingsFile)
filtIdx = len(planC.scan)-1         # Index of filtered scan

In [6]:
# Visualize in Napari 
dispScanNum = [scanNum,filtIdx]
dispStrNum = [strNum]
dispDoseNum = []
viewer, scan_layer, struct_layer, dose_layer, dvf_layer = \
    vwr.showNapari(planC, scan_nums=dispScanNum, struct_nums=dispStrNum, dose_nums=[], vectors_dict={}, displayMode='2d')


## Example-2 : 3D LoG filter

In [7]:
#Compute filter response 
settingsFile =  'C:\settings\convolutional_filter_settings\LoG_filter.json' #Replace with Path to sample loG filter settings file
planC = textureUtils.generateTextureMapFromPlanC(planC, scanNum, strNum, settingsFile)
print(len(planC.scan))

#Visualize in Napari 
filtIdx = len(planC.scan)-1
dispScanNum = [scanNum,filtIdx]
viewer, scan_layer, struct_layer, dose_layer, dvf_layer = \
    vwr.showNapari(planC, scan_nums=dispScanNum, struct_nums=dispStrNum, dose_nums=[], vectors_dict={}, displayMode='2d')


## Example-3: Rotation-invariant Laws' energy filter

In [8]:
#Compute filter response
settingsFile = 'C:\settings\convolutional_filter_settings\Rot_inv_laws_energy_filter.json' #Replace with Path to sample Laws' filter settings file
planC = textureUtils.generateTextureMapFromPlanC(planC, scanNum, strNum, settingsFile)

#Display 
filtIdx = len(planC.scan)-1
dispScanNum = [scanNum,filtIdx]
viewer, scan_layer, struct_layer, dose_layer, dvf_layer = \
    vwr.showNapari(planC, scan_nums=dispScanNum, struct_nums=dispStrNum, dose_nums=[], vectors_dict={}, displayMode='2d')


## Example-4: 3D approximation of Gabor filter

In [9]:
#Compute filter response
settingsFile = 'C:\settings\convolutional_filter_settings\gabor_filter.json' #Replace with Path to sample Gabor filter settings file
planC = textureUtils.generateTextureMapFromPlanC(planC, scanNum, strNum, settingsFile)

#Visualize in Napari 
filtIdx = len(planC.scan)-1
dispScanNum = [scanNum,filtIdx]
viewer, scan_layers, struct_layer, dose_layers, dvf_layer = \
    vwr.showNapari(planC, scan_nums=dispScanNum, struct_nums=dispStrNum, dose_nums=[], vectors_dict={}, displayMode='2d')
